# Computational Appendix

This notebook is an attempt at creating Julia version of codes in the [computational appendix](http://jbrwww.che.wisc.edu/home/jbraw/chemreacfun/web-appendix.pdf) of the book [Chemical Reactor Analysis and Design by Rawlings and Ekerdt](http://jbrwww.che.wisc.edu/home/jbraw/chemreacfun/)

In [1]:
using Gadfly

## A.1 Linear Algebra and Least Squares

In [2]:
stoi = [0 1 0 -1 -1 1;
        -1 1 1 -1 0 0;
         1 0 -1 0 -1 1]

3×6 Array{Int64,2}:
  0  1   0  -1  -1  1
 -1  1   1  -1   0  0
  1  0  -1   0  -1  1

In [3]:
rank(stoi)

2

In [4]:
r = [1;2;3]

3-element Array{Int64,1}:
 1
 2
 3

In [5]:
R = stoi' * r

6-element Array{Int64,1}:
  1
  3
 -1
 -3
 -4
  4

### Example A.1

In [6]:
stoi = [0 1 0 -1 -1 1; -1 1 1 -1 0 0]

2×6 Array{Int64,2}:
  0  1  0  -1  -1  1
 -1  1  1  -1   0  0

In [7]:
nr, nspec = size(stoi)

(2,6)

In [8]:
r = [1;2]

2-element Array{Int64,1}:
 1
 2

In [9]:
R = stoi' * r

6-element Array{Int64,1}:
 -2
  3
  2
 -3
 -1
  1

In [10]:
nmeas = 2000
R_meas = zeros(nspec,nmeas)
for i in 1:nmeas
    R_meas[:,i] = 0.05 * randn(nspec) + R ;
end

In [11]:
R_meas

6×2000 Array{Float64,2}:
 -2.0039    -1.91629   -2.0718   -1.95138  …  -1.95285   -2.02694   -1.93315
  2.99482    3.07156    2.97288   2.95766      2.9316     3.00617    3.04766
  2.03864    2.03632    1.98307   1.99051      1.95148    1.97101    1.98355
 -3.00012   -3.0198    -3.11866  -3.02951     -2.94884   -2.95559   -3.00638
 -1.02207   -1.01582   -1.07383  -1.01293     -0.994833  -0.994512  -1.06399
  0.963977   0.885746   1.0049    1.02687  …   1.0222     0.980828   1.0557 

In [12]:
r_est = stoi' \ R_meas

2×2000 Array{Float64,2}:
 0.987417  0.990314  1.03235  1.02082  1.05987  …  1.00169  0.985749  1.06279
 2.01566   2.01583   2.02043  1.97186  1.96021     1.94535  1.99705   1.96129

In [13]:
plot(x = r_est[1,:], y = r_est[2,:], Guide.xlabel("r1"), Guide.ylabel("r2"))

ErrorException: error compiling #call#112: error compiling Type: could not load library "/Users/shanki/.julia/v0.5/Homebrew/deps/usr/lib/libpangocairo-1.0.dylib"
dlopen(/Users/shanki/.julia/v0.5/Homebrew/deps/usr/lib/libpangocairo-1.0.dylib, 1): Library not loaded: /Users/shanki/.julia/v0.5/Homebrew/deps/usr/opt/harfbuzz/lib/libharfbuzz.0.dylib
  Referenced from: /Users/shanki/.julia/v0.5/Homebrew/deps/usr/lib/libpangocairo-1.0.dylib
  Reason: image not found

## A.2 Nonlinear Algebraic Equations

In [14]:
function dgdx!(x, residual)
    K1 = 108
    K2 = 284
    P = 2.5
    yI0 = 0.5
    yB0 = 0.5
    yP10 = 0.0
    yP20 = 0.0
    d = 1 - x[1] - x[2]
    yI = (yI0 - x[1] - x[2])/d
    yB = (yB0 - x[1] - x[2])/d
    yP1 = (yP10 + x[1])/d
    yP2 = (yP20 + x[2])/d
    
    residual[1] = P*K1*yI*yB - yP1
    residual[2] = P*K2*yI*yB - yP2
    
end

dgdx! (generic function with 1 method)

In [15]:
# initial guess
x0 = [0.2, 0.2]

2-element Array{Float64,1}:
 0.2
 0.2

In [16]:
using NLsolve

In [17]:
nlsolve(dgdx!,x0)

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in slice(::Array{Float64,2}, ::Vararg{Any,N}) at ./deprecated.jl:30
 in sumabs2j at /Users/shanki/.julia/v0.5/NLsolve/src/utils.jl:1 [inlined]
 in trust_region_(::NLsolve.DifferentiableMultivariateFunction, ::Array{Float64,1}, ::Float64, ::Float64, ::Int64, ::Bool, ::Bool, ::Bool, ::Float64, ::Bool) at /Users/shanki/.julia/v0.5/NLsolve/src/trust_region.jl:105
 in #nlsolve#17(::Symbol, ::Float64, ::Float64, ::Int64, ::Bool, ::Bool, ::Bool, ::Function, ::Float64, ::Bool, ::NLsolve.#nlsolve, ::NLsolve.DifferentiableMultivariateFunction, ::Array{Float64,1}) at /Users/shanki/.julia/v0.5/NLsolve/src/nlsolve_func_defs.jl:24
 in (::NLsolve.#kw##nlsolve)(::Array{Any,1}, ::NLsolve.#nlsolve, ::NLsolve.DifferentiableMultivariateFunction, ::Array{Float64,1}) at ./<missing>:0
 in #nlsolve#19(::Symbol, ::Float64, ::Float64, ::Int64, ::Bool, ::Bool, ::Bool, ::Function, ::Float64, ::Bool, ::Bool, ::NLsolve.#nlsolve, ::#dgdx!, ::Array{Float64,1}) at

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.2,0.2]
 * Zero: [0.133357,0.350679]
 * Inf-norm of residuals: 0.000000
 * Iterations: 6
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 7
 * Jacobian Calls (df/dx): 7

In [18]:
using Sundials

In [19]:
res = Sundials.kinsol(dgdx!, x0)

2-element Array{Float64,1}:
 0.133357
 0.350679

In [20]:
using Ipopt

In [21]:
function gibbs(x) 
    
    K1 = 108.0
    K2 = 284.0
    P = 2.5
    yI0 = 0.5
    yB0 = 0.5
    yP10 = 0.0
    yP20 = 0.0
    d = 1 - x[1] - x[2]
    yI = (yI0 - x[1] - x[2])/d
    yB = (yB0 - x[1] - x[2])/d
    yP1 = (yP10 + x[1])/d
    yP2 = (yP20 + x[2])/d
    
    gibbsval = -(x[1]*log(K1)+x[2]*log(K2))+(1.0-x[1]-x[2])*log(P)+yB*d*log(yB)+yP1*d*log(yP1)+yP2*d*log(yP2)+yI*d*log(yI)
    
#    println(x," ",gibbsval)
    return gibbsval
end

gibbs (generic function with 1 method)

In [22]:
function gibbs_grad(x, grad_f)
    
    K1 = 108.0
    K2 = 284.0
    P = 2.5
    yI0 = 0.5
    yB0 = 0.5
    yP10 = 0.0
    yP20 = 0.0
    d = 1 - x[1] - x[2]
    yI = (yI0 - x[1] - x[2])/d
    yB = (yB0 - x[1] - x[2])/d
    yP1 = (yP10 + x[1])/d
    yP2 = (yP20 + x[2])/d
    
    grad_f[1] = -log(K1)-log(P)-log(yI)+d*(-1.0/d+yI/d)-log(yB)+d*(-1.0/d+yB/d)+log(yP1)+d*(1.0/d+yP1/d)+d*(yP2/d)
    grad_f[2] = -log(K2)-log(P)-log(yI)+d*(-1.0/d+yI/d)-log(yB)+d*(-1.0/d+yB/d)+d*(yP1/d)+log(yP2)+d*(1.0/d+yP2/d)
    println("manual gradient")
    println(grad_f)
    println("ad version")
    println(ForwardDiff.gradient(gibbs,x))
end  

gibbs_grad (generic function with 1 method)

In [23]:
function eval_cons(x, g)
    g[1] = -x[1] - x[2]
end

eval_cons (generic function with 1 method)

In [24]:
function eval_jac_cons(x, mode, rows, cols, values)
    if mode == :Structure
        rows[1] = 1; cols[1] = 1
        rows[2] = 1; cols[2] = 2
    else
        values[1] = -1.0
        values[2] = -1.0
    end
end

eval_jac_cons (generic function with 1 method)

In [25]:
function eval_hess(x, mode, rows, cols, obj_factor, lambda, values)
    
    K1 = 108.0
    K2 = 284.0
    P = 2.5
    yI0 = 0.5
    yB0 = 0.5
    yP10 = 0.0
    yP20 = 0.0
    d = 1 - x[1] - x[2]
    yI = (yI0 - x[1] - x[2])/d
    yB = (yB0 - x[1] - x[2])/d
    yP1 = (yP10 + x[1])/d
    yP2 = (yP20 + x[2])/d    
    
    h00 = (-1.0/yI)*(-1.0/d+yI/d)-1.0/d+yI/d-(1.0/yB)*(-1.0/d+yB/d)-1.0/d+yB/d+(1.0/yP1)*(1.0/d+yP1/d)+(1.0/d+yP1/d)+yP2/d 
    h10 = (-1.0/yI)*(-1.0/d+yI/d)-1.0/d+yI/d-(1.0/yB)*(-1.0/d+yB/d)-1.0/d+yB/d+(1.0/yP1)*(yP1/d)+yP1/d+1.0/d+yP2/d 
    h11 = (-1.0/yI)*(-1.0/d+yI/d)-1.0/d+yI/d-(1.0/yB)*(-1.0/d+yB/d)-1.0/d+yB/d+yP1/d+(1.0/yP2)*(1.0/d+yP2/d)+(1.0/d+yP2/d)
    
    if mode == :Structure
        idx = 1
        for row = 1:2
            for col = 1:row
                rows[idx] = row
                cols[idx] = col
                idx += 1
            end
        end
    else
        values[1] = obj_factor*h00
        values[2] = obj_factor*h10
        values[3] = obj_factor*h11
    end
end

eval_hess (generic function with 1 method)

In [26]:
function gibbs_grad_ad(x, grad_f)
    
    tmp = ForwardDiff.gradient(gibbs,x)
    grad_f[1] = tmp[1]
    grad_f[2] = tmp[2]
    println(grad_f)
end  

gibbs_grad_ad (generic function with 1 method)

In [27]:
function eval_hess_ad(x, mode, rows, cols, obj_factor, lambda, values)   
    if mode == :Structure
        idx = 1
        for row = 1:2
            for col = 1:row
                rows[idx] = row
                cols[idx] = col
                idx += 1
            end
        end
    else
        hess = ForwardDiff.hessian(gibbs,x)
        #println(hess)
        values[1] = obj_factor*hess[1,1]
        values[2] = obj_factor*hess[2,1]
        values[3] = obj_factor*hess[2,2]
    end
end

eval_hess_ad (generic function with 1 method)

In [28]:
n = 2
m = 1
x_L = [0.0, 0.0]
x_U = [0.5, 0.5]
g_L = [-0.499]
g_U = [0.0]
nnz_jac = m*n
nnz_hes = 3

3

In [29]:
prob = createProblem(n, x_L, x_U, m, g_L, g_U, nnz_jac, nnz_hes,
gibbs, eval_cons, gibbs_grad, eval_jac_cons, eval_hess)

Ipopt.IpoptProblem(Ptr{Void} @0x00007fb896fc5d70,2,1,[0.0,0.0],[0.0],[0.0],[0.0,0.0],[0.0,0.0],0.0,0,gibbs,eval_cons,gibbs_grad,eval_jac_cons,eval_hess,nothing,:Min)

In [30]:
# Set starting solution
prob.x = [0.2,0.2]

#addOption(prob, "hessian_approximation", "limited-memory")

# Solve
status = solveProblem(prob)

manual gradient
[-3.11352,-4.08036]
ad version
[-3.11352,-4.08036]
manual gradient
[-3.11352,-4.08036]
ad version
[-3.11352,-4.08036]

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
              

0

In [31]:
prob.x

2-element Array{Float64,1}:
 0.133357
 0.350679

In [32]:
prob_ad = createProblem(n, x_L, x_U, m, g_L, g_U, nnz_jac, nnz_hes, gibbs, eval_cons, 
gibbs_grad_ad, eval_jac_cons, eval_hess_ad)

Ipopt.IpoptProblem(Ptr{Void} @0x00007fb895fd1010,2,1,[0.0,0.0],[0.0],[0.0],[0.0,0.0],[0.0,0.0],0.0,0,gibbs,eval_cons,gibbs_grad_ad,eval_jac_cons,eval_hess_ad,nothing,:Min)

In [33]:
# Set starting solution
prob_ad.x = [0.2,0.2]

# Solve
status = solveProblem(prob_ad)

[-3.11352,-4.08036]
[-3.11352,-4.08036]
This is Ipopt version 3.12.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        1
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||

0

In [34]:
prob_ad.x

2-element Array{Float64,1}:
 0.133357
 0.350679

In [35]:
using NLopt

In [36]:
function gibbs_nlopt(x::Vector, grad::Vector)
    if length(grad) > 0
        tmp = ForwardDiff.gradient(gibbs, x)
        grad[1] = tmp[1]
        grad[2] = tmp[2]
    end

    fval = gibbs(x) 
    
    return fval
end

gibbs_nlopt (generic function with 1 method)

In [37]:
function cons_nlopt(x::Vector, grad::Vector)
    if length(grad) > 0
        grad[1] = 1.0
        grad[2] = 1.0
    end
    x[1] + x[2] - 0.499
end

cons_nlopt (generic function with 1 method)

In [41]:
opt = Opt(:LD_MMA, 2)
lower_bounds!(opt, [0., 0.])
upper_bounds!(opt, [0.5, 0.5])
xtol_rel!(opt,1e-4)

min_objective!(opt, gibbs_nlopt)
inequality_constraint!(opt, (x,g) -> cons_nlopt(x,g), 1e-8)

(minf,minx,ret) = optimize(opt, [0.2, 0.2])

(-2.559423417560121,[0.133679,0.350366],:XTOL_REACHED)